In [1]:
import pygame
pygame.init()  
import numpy as np
import math

pygame 2.3.0 (SDL 2.24.2, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Tela(object):
    def __init__(self, title, bgColor, width, height):
        self.title = title       
        self.bgColor = bgColor  
        self.width = width      
        self.height = height    
        self.tela = pygame.display.set_mode(self.obterTamanho()) 
        pygame.display.set_caption(self.title)           
        self.clock = pygame.time.Clock()
        
    def executar(self, obj):
        while True:  
            for event in pygame.event.get(): 
                if event.type == pygame.QUIT:
                    return pygame.quit()
            
            self.tela.fill(self.bgColor)
            obj.draw(self);
            pygame.display.update()
            self.clock.tick(50)
        
    def obterTamanho(self):
        return (self.width, self.height)
    
    def definirPixel(self, x, y, color):
        self.tela.set_at((x, y), color)

In [3]:
class Primitiva(object):
    def __init__(self, id):
        self.id = id

In [4]:
class Line(Primitiva):
    def __init__(self, x1, y1, x2, y2, color):
        self.x1 = x1       
        self.x2 = x2       
        self.y1 = y1       
        self.y2 = y2       
        self.color = color 
    
    def draw(self, tela):
        self.dda(tela)


    def dda(self, tela):      
        dx, dy, k = 0, 0, 0
        x_inc, y_inc = 0.0, 0.0
        x, y = 0.0, 0.0
    
        dx = self.x2 - self.x1
        dy = self.y2 - self.y1
    
        if abs(dx) > abs(dy):
            iter = abs(dx)
        else:
            iter = abs(dy)
        
        x_inc = dx/iter
        y_inc = dy/iter

        x = self.x1
        y = self.y1

        tela.definirPixel(round(x), round(y), self.color)

        for k in range(iter):
            x = x + x_inc
            y = y + y_inc
            
            tela.definirPixel(round(x), round(y), self.color)
            
    def getListOfPoints(self):
        l = []
        l.append(Point3D(self.x1, self.y1, 0))
        l.append(Point3D(self.x2, self.y2, 0))
        return l
    
    def transform(self, T):
        point1 = np.array([self.x1, self.y1, 0, 1]).T
        point2 = np.array([self.x2, self.y2, 0, 1]).T
        res1 = T.matrix.dot(point1)
        res2 = T.matrix.dot(point2)
        l = Line(int(round(res1[0])), int(round(res1[1])), int(round(res2[0])), int(round(res2[1])), self.color)     
        return l

In [5]:
class Circle(Primitiva):
    def __init__(self, xc, yc, raio, color):
        self.xc = xc     
        self.yc = yc      
        self.raio = raio   
        self.color = color 
    
    def draw(self, tela):
        x = 0               
        y = self.raio       
        d = 1 - self.raio    
        self.drawCirclePoints(x, y, tela)     
        while x < y:
            if d < 0: 
                d = d + 2 * x + 3
            else: 
                d = d + 2 * (x - y) + 5
                y = y - 1
            x = x + 1
            self.drawCirclePoints(x, y, tela)
            
    def drawCirclePoints(self, x, y, tela):
        xCentro = self.xc
        yCentro = self.yc
        tela.definirPixel(xCentro + x, yCentro + y, self.color)
        tela.definirPixel(xCentro + y, yCentro + x, self.color)
        tela.definirPixel(xCentro + y, yCentro - x, self.color)
        tela.definirPixel(xCentro + x, yCentro - y, self.color)
        tela.definirPixel(xCentro - x, yCentro - y, self.color)
        tela.definirPixel(xCentro - y, yCentro - x, self.color)
        tela.definirPixel(xCentro - y, yCentro + x, self.color)
        tela.definirPixel(xCentro - x, yCentro + y, self.color)

    def transform(self, T):
        centerPoint = np.array([self.xc, self.yc, 0, 1]).T
        res = T.matrix.dot(centerPoint)
        c = Circle(int(round(res[0])), int(round(res[1])), self.raio, self.color)
        return c

In [6]:
class Point2D(object):
    def __init__(self, x, y, color):
        self.x = x
        self.y = y
        self.color = color
    
    def draw(self, tela):
        tela.definirPixel(self.x, self.y, self.color)     

In [7]:
class GeometricTransformation(object):
    def __init__(self, matrix):
        self.matrix = matrix

class Translation3D(GeometricTransformation):
    def __init__(self, Tx, Ty, Tz):
        super().__init__(np.array([[1, 0, 0, Tx], [0, 1, 0, Ty], [0, 0, 1, Tz], [0, 0, 0, 1]]))
        
class Rotation3D(GeometricTransformation):
    def __init__(self, theta, axis):
        self.axis = axis
        if axis == 'z':
            super().__init__(np.array([[math.cos(theta), -math.sin(theta), 0, 0], [math.sin(theta), math.cos(theta), 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]))
        elif axis == 'x':
            super().__init__(np.array([[1, 0, 0, 0], [0, math.cos(theta), -math.sin(theta), 0], [0, math.sin(theta), math.cos(theta), 0], [0, 0, 0, 1]]))
        elif axis == 'y':
            super().__init__(np.array([[math.cos(theta), 0, math.sin(theta), 0], [0, 1, 0, 0], [-math.sin(theta), 0, math.cos(theta), 0], [0, 0, 0, 1]]))
        else:
            exit()

class CombinedTransformation(GeometricTransformation):
    def __init__(self):
        super().__init__([])
        self.stack = []
    
    def add(self, matrix):
        self.stack.append(matrix)
              
    def combine(self):
        while (len(self.stack) > 1):
            m2 = self.stack.pop(-1)
            m1 = self.stack.pop(-1)
            m3 = GeometricTransformation(m2.matrix.dot(m1.matrix))
            self.stack.append(m3)
            
        self.matrix = self.stack.pop(0).matrix
              
class Point3D(Point2D):
    def __init__(self, x, y, z, color):
        super().__init__(x, y, color)
        self.z = z 
                      
class RotationInArbitraryAxis(CombinedTransformation):
    def __init__(self, pointA, pointB, theta):
        super().__init__()
        super().add(Translation3D(-pointA.x, -pointA.y, -pointA.z))
        super().add(Rotation3D(theta, 'z'))
        super().add(Translation3D(pointA.x, pointA.y, pointA.z))
        super().combine()

In [8]:
class Chute(object):
    def __init__(self):
        self.primitivas = []
        self.count = 0
        self.bolax = 0
        self.bolay = 0

    def draw(self, tela):
        quadril = Line(120, 100, 260, 270, pygame.Color(0, 0, 0, 255))
        joelho = Line(260, 270, 180, 500, pygame.Color(0, 255, 0, 255))
        tornozelo = Line(180, 500, 230, 620, pygame.Color(255, 140, 0, 255))
        bola = Circle(300, 480, 50, pygame.Color(0, 0, 255, 255))

        quadril_rotacao = RotationInArbitraryAxis(
            Point3D(120, 100, 10, pygame.Color(0)),
            Point3D(260, 270, 10, pygame.Color(0)),
            math.pi * self.count / 200
        )
        quadril_transformado = quadril.transform(quadril_rotacao)

        joelho_rotacao = RotationInArbitraryAxis(
            Point3D(quadril_transformado.x1, quadril_transformado.y1, 0, pygame.Color(0)),
            Point3D(quadril_transformado.x2, quadril_transformado.y2, 0, pygame.Color(0)),
            math.pi * self.count / 200
        )
        joelho_transformado = joelho.transform(joelho_rotacao)

        tornozelo_rotacao = RotationInArbitraryAxis(
            Point3D(quadril_transformado.x1, quadril_transformado.y1, 0, pygame.Color(0)),
            Point3D(quadril_transformado.x2, quadril_transformado.y2, 0, pygame.Color(0)),
            math.pi * self.count / 200
        )
        tornozelo_transformado = tornozelo.transform(tornozelo_rotacao)

        self.count -= 1
        if self.count <= -10:
            self.bolax += 50
            self.bolay -= 20
            bola_rotacao = Translation3D(self.bolax, self.bolay, 0)
            bola_transformada = bola.transform(bola_rotacao)
        
        else:bola_transformada = bola

        if self.count <= -30:
            self.count = 10
            self.bolax = 20
            self.bolay = 20

        self.primitivas.append(quadril_transformado)
        self.primitivas.append(joelho_transformado)
        self.primitivas.append(tornozelo_transformado)
        self.primitivas.append(bola_transformada)

        for point in self.primitivas:
            point.draw(tela)

        self.primitivas.clear()

In [ ]:
tela = Tela("Animação chute", pygame.Color(255, 255, 255, 255), 800, 800)

animacao = Chute();

tela.executar(animacao);